In [4]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


###############################################
# Feature engineering: build enriched features
###############################################
def build_enriched_feature_set(pop_path = '../raw_data/world_population.csv', wb_path = '../raw_data/WorldBank.csv'):
    """
    Load world population + World Bank data and build the enriched feature set
    used in the main modeling script.

    Returns:
        X_enriched : DataFrame of features
        y          : numpy array of log(2020 population)
        meta       : DataFrame with identifiers (Country Name, CCA3, Region, IncomeGroup)
        df         : DataFrame with raw merged data (includes 2020 Population)
    """
    # Load the raw data
    pop = pd.read_csv(pop_path)
    wb = pd.read_csv(wb_path)

    # Indicators we use from World Bank
    indicator_cols = [
        'Birth rate, crude (per 1,000 people)',
        'Death rate, crude (per 1,000 people)',
        'Electric power consumption (kWh per capita)',
        'GDP (USD)',
        'GDP per capita (USD)',
        'Individuals using the Internet (% of population)',
        'Infant mortality rate (per 1,000 live births)',
        'Life expectancy at birth (years)',
        'Population density (people per sq. km of land area)',
        'Unemployment (% of total labor force) (modeled ILO estimate)'
    ]

    # Slice World Bank data for 2010 and 2000
    wb_2010 = wb[wb['Year'] == 2010][['Country Code', 'Country Name', 'Region', 'IncomeGroup'] + indicator_cols].copy()
    wb_2000 = wb[wb['Year'] == 2000][['Country Code'] + indicator_cols].copy()

    # Rename 2000 columns so we can compute deltas
    rename_cols = {col: col + ' 2000' for col in indicator_cols}
    wb_2000 = wb_2000.rename(columns = rename_cols)

    # Merge population data with 2010 indicators
    df = pop.merge(
        wb_2010,
        left_on = 'CCA3',
        right_on = 'Country Code',
        how = 'inner'
    )
    # Merge 2000 indicators
    df = df.merge(
        wb_2000,
        on = 'Country Code',
        how = 'left'
    )

    # Reset index to keep everything aligned (0..n-1)
    df = df.reset_index(drop = True)

    # Target: log(2020 population)
    y = np.log(df['2020 Population'].values)

    # Baseline: log(2010 population) + region dummies
    X_baseline = pd.DataFrame({
        'log_pop_2010': np.log(df['2010 Population'].values)
    })
    X_baseline = pd.concat(
        [X_baseline, pd.get_dummies(df['Region'], prefix = 'Region', drop_first = True)],
        axis = 1
    )

    # Demo + econ features (levels in 2010)
    X_demo = X_baseline.copy()
    X_demo['birth_rate_2010'] = df['Birth rate, crude (per 1,000 people)']
    X_demo['life_expectancy_2010'] = df['Life expectancy at birth (years)']
    X_demo['log_gdp_pc_2010'] = np.log(df['GDP per capita (USD)'].replace(0, np.nan))
    X_demo['log_density_2010'] = np.log(df['Population density (people per sq. km of land area)'].replace(0, np.nan))
    X_demo = pd.concat(
        [X_demo, pd.get_dummies(df['IncomeGroup'], prefix = 'Income', drop_first = True)],
        axis = 1
    )

    # Enriched feature set: add trend and infrastructure variables
    X_enriched = X_demo.copy()

    # Growth in population 2000–2010
    pop_2000 = df['2000 Population']
    pop_2010 = df['2010 Population']
    ratio = pop_2010 / pop_2000.replace(0, np.nan)
    X_enriched['growth_00_10'] = ratio.replace([np.inf, -np.inf], np.nan)

    # Changes in key indicators 2000–2010
    X_enriched['delta_birth_rate_00_10'] = (
        df['Birth rate, crude (per 1,000 people)'] -
        df['Birth rate, crude (per 1,000 people) 2000']
    )
    X_enriched['delta_life_exp_00_10'] = (
        df['Life expectancy at birth (years)'] -
        df['Life expectancy at birth (years) 2000']
    )
    X_enriched['delta_gdp_pc_00_10'] = (
        df['GDP per capita (USD)'] -
        df['GDP per capita (USD) 2000']
    )

    # 2010 infrastructure / development variables
    X_enriched['electricity_2010'] = df['Electric power consumption (kWh per capita)']
    X_enriched['internet_2010'] = df['Individuals using the Internet (% of population)']
    X_enriched['unemployment_2010'] = df['Unemployment (% of total labor force) (modeled ILO estimate)']

    # Meta columns for later lookup and stratification
    meta = df[['Country Name', 'CCA3', 'Region', 'IncomeGroup']].copy()

    return X_enriched, y, meta, df


###############################################
# Define the final stacked model
###############################################
def build_final_model():
    """
    Build the Stack_Lasso_RF model:
    - Lasso + RandomForest as base learners
    - Ridge as the meta-learner
    """
    # Base learner 1: Lasso pipeline (impute + scale + Lasso)
    lasso_pipe = Pipeline([
        ('imputer', SimpleImputer(strategy = 'median')),
        ('scaler', StandardScaler()),
        ('reg', Lasso(alpha = 0.001, max_iter = 10000))
    ])

    # Base learner 2: Random Forest pipeline (impute + RF)
    rf_pipe = Pipeline([
        ('imputer', SimpleImputer(strategy = 'median')),
        ('rf', RandomForestRegressor(
            n_estimators = 500,
            max_depth = None,
            min_samples_leaf = 1,
            max_features = 0.5,
            random_state = 0,
            n_jobs = -1
        ))
    ])

    # Meta-learner on top: Ridge regression
    final_reg = Ridge(alpha = 0.1)

    # StackingRegressor that combines Lasso and RF
    stack_model = StackingRegressor(
        estimators = [
            ('lasso', lasso_pipe),
            ('rf', rf_pipe)
        ],
        final_estimator = final_reg,
        n_jobs = -1
    )

    return stack_model


###############################################
# Cross-validation evaluation of the model
###############################################
def evaluate_model_cv(X, y, meta, n_splits = 5):
    """
    Run k-fold cross-validation for the stacked model on the enriched feature set.

    For each fold, we:
      - build a fresh stacked model,
      - train on the training folds,
      - evaluate on the validation fold (RMSE, MAE, R2).

    At the end, we print the mean and std of each metric across folds.
    """
    strat_labels = meta['IncomeGroup'].fillna('Unknown').values

    skf = StratifiedKFold(
        n_splits = n_splits,
        shuffle = True,
        random_state = 0
    )

    rmse_list = []
    mae_list = []
    r2_list = []

    print(f"\nRunning {n_splits}-fold cross-validation for the stacked model...\n")

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, strat_labels), start = 1):
        # Build a fresh model for this fold
        model = build_final_model()

        X_train = X.iloc[train_idx]
        y_train = y[train_idx]
        X_val = X.iloc[val_idx]
        y_val = y[val_idx]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)

        rmse = mean_squared_error(y_val, y_pred, squared = False)
        mae = mean_absolute_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)

        rmse_list.append(rmse)
        mae_list.append(mae)
        r2_list.append(r2)

        print(f"Fold {fold}: RMSE = {rmse:.3f}, MAE = {mae:.3f}, R2 = {r2:.3f}")

    print("\n=== Cross-validation summary ===")
    print(f"RMSE: mean = {np.mean(rmse_list):.3f}, std = {np.std(rmse_list):.3f}")
    print(f"MAE:  mean = {np.mean(mae_list):.3f}, std = {np.std(mae_list):.3f}")
    print(f"R2:   mean = {np.mean(r2_list):.5f}, std = {np.std(r2_list):.5f}")
    print("")


###############################################
# Helper: look up a country by name or CCA3 code
###############################################
def find_country_index(meta, query):
    """
    Given a meta DataFrame and a user query, try to find the row index
    of the matching country.

    We first try:
    - exact match on CCA3 code (case-insensitive)
    - exact match on country name (case-insensitive, trimmed)

    If that fails, we try a "contains" match on the country name
    and return the first hit.

    Returns:
        idx (int) if found, or None if not found.
    """
    if not query:
        return None

    q = query.strip()

    # Try match by 3-letter country code (CCA3)
    if len(q) == 3:
        mask_code = meta['CCA3'].str.upper() == q.upper()
        if mask_code.any():
            return int(np.where(mask_code.values)[0][0])

    # Try exact match on country name (case-insensitive)
    name_series = meta['Country Name'].str.strip().str.lower()
    mask_name = name_series == q.lower()
    if mask_name.any():
        return int(np.where(mask_name.values)[0][0])

    # Try partial match on country name
    contains_mask = name_series.str.contains(q.lower())
    if contains_mask.any():
        possible = meta[contains_mask]['Country Name'].tolist()
        print("No exact match found. Found similar name(s):")
        for name in possible:
            print("  -", name)
        # For simplicity, pick the first match
        first_idx = int(np.where(contains_mask.values)[0][0])
        print("Using:", meta.iloc[first_idx]['Country Name'])
        return first_idx

    return None


###############################################
# Main: optional evaluation + interactive prediction
###############################################
if __name__ == '__main__':
    # 1. Load data and build features
    print("Loading data and building enriched feature set...")
    X_enriched, y, meta, df = build_enriched_feature_set()
    print(f"Data loaded. Number of countries: {X_enriched.shape[0]}\n")

    # 2. Ask user if they want to run k-fold evaluation
    eval_choice = input("Do you want to run a k-fold cross-validation evaluation now? (y/n): ").strip().lower()
    if eval_choice in ['y', 'yes']:
        evaluate_model_cv(X_enriched, y, meta, n_splits = 5)

    # 3. Train the final stacked model on all countries
    print("Training stacked model on all countries...")
    model = build_final_model()
    model.fit(X_enriched, y)
    print("Training complete.\n")

    # 4. Interactive prediction loop
    print("You can now query predictions for a country.")
    print("Type a country name (e.g., 'France') or 3-letter code (e.g., 'FRA').")
    print("Type 'q' or 'quit' to exit.\n")

    while True:
        user_input = input("Enter country name or 3-letter code (q to quit): ").strip()
        if user_input.lower() in ['q', 'quit', 'exit']:
            print("Exiting.")
            break

        idx = find_country_index(meta, user_input)
        if idx is None:
            print("Country not found. Please try again.\n")
            continue

        # Get single-row feature matrix for this country
        X_row = X_enriched.iloc[[idx]]  # keep as DataFrame with one row

        # Predict log population and convert back to level
        log_pred = model.predict(X_row)[0]
        pred_pop = float(np.exp(log_pred))

        # Get actual population from df
        actual_pop = float(df.iloc[idx]['2020 Population'])
        country_name = meta.iloc[idx]['Country Name']
        cca3 = meta.iloc[idx]['CCA3']

        # Compute percentage error
        if actual_pop > 0:
            pct_error = 100.0 * (pred_pop - actual_pop) / actual_pop
        else:
            pct_error = np.nan

        print(f"\nCountry: {country_name} ({cca3})")
        print(f"Predicted 2020 population: {pred_pop:,.0f}")
        print(f"Actual 2020 population:    {actual_pop:,.0f}")
        if not np.isnan(pct_error):
            print(f"Percentage error:          {pct_error:+.2f}%")
        print("")


Loading data and building enriched feature set...
Data loaded. Number of countries: 209


Running 5-fold cross-validation for the stacked model...



TypeError: got an unexpected keyword argument 'squared'